In [1]:
from IPython.display import display, Markdown
from malid import config, logger
from malid.external import model_evaluation
from malid.trained_model_wrappers import SequenceClassifier
from malid.datamodels import (
    TargetObsColumnEnum,
    SampleWeightStrategy,
    combine_classification_option_names,
)

sample_weight_strategy = SampleWeightStrategy.ISOTYPE_USAGE

In [2]:
for gene_locus in config.gene_loci_used:
    for target_obs_column in config.classification_targets:
        models_base_dir = SequenceClassifier._get_model_base_dir(
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
            sample_weight_strategy=sample_weight_strategy,
        )  # should already exist
        model_location = models_base_dir / "train_smaller_model"

        output_dir = (
            config.paths.sequence_models_output_dir
            / gene_locus.name
            / combine_classification_option_names(
                target_obs_column=target_obs_column,
                sample_weight_strategy=sample_weight_strategy,
            )
        )  # might not yet exist
        output_dir.mkdir(exist_ok=True, parents=True)  # create if needed

        try:
            logger.info(
                f"{gene_locus}, {target_obs_column}, {sample_weight_strategy} -> {model_location} -> {output_dir}"
            )

            ## Load and summarize
            experiment_set = model_evaluation.ExperimentSet.load_from_disk(
                output_prefix=model_location
            )

            # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
            # TODO: make kdict support: del self.model_outputs[:, fold_id]
            for key in experiment_set.model_outputs[:, -1].keys():
                logger.debug(f"Removing {key} (global fold)")
                del experiment_set.model_outputs[key]

            experiment_set_global_performance = experiment_set.summarize()
            experiment_set_global_performance.export_all_models(
                func_generate_classification_report_fname=lambda model_name: output_dir
                / f"sequence_model.results_on_validation_set.{model_name}.classification_report.txt",
                func_generate_confusion_matrix_fname=lambda model_name: output_dir
                / f"sequence_model.results_on_validation_set.{model_name}.confusion_matrix.png",
                dpi=72,
            )
            combined_stats = (
                experiment_set_global_performance.get_model_comparison_stats()
            )
            combined_stats.to_csv(
                output_dir
                / "sequence_model.results_on_validation_set.compare_model_scores.tsv",
                sep="\t",
            )

            # Display
            display(
                Markdown(
                    f"# {gene_locus}, {target_obs_column}, {sample_weight_strategy}"
                )
            )
            display(combined_stats)

        except Exception as err:
            logger.exception(
                f"Failed to analyze {gene_locus}, {target_obs_column}, {sample_weight_strategy}: {err}"
            )

        print("*" * 80)

2023-01-03 19:43:15,050 - analyze_sequence_model.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease, SampleWeightStrategy.ISOTYPE_USAGE -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/sequence_models/BCR/disease_sample_weight_strategy_ISOTYPE_USAGE/train_smaller_model -> /users/maximz/code/boyd-immune-repertoire-classification/out/unirep_fine_tuned/sequence_models/BCR/disease_sample_weight_strategy_ISOTYPE_USAGE


# GeneLocus.BCR, TargetObsColumnEnum.disease, SampleWeightStrategy.ISOTYPE_USAGE

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,Accuracy with sample weights per fold,MCC per fold,MCC with sample weights per fold,Accuracy global,Accuracy with sample weights global,MCC global,MCC with sample weights global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_multiclass,0.598 +/- 0.013 (in 3 folds),0.606 +/- 0.016 (in 3 folds),0.574 +/- 0.009 (in 3 folds),0.582 +/- 0.013 (in 3 folds),0.312 +/- 0.011 (in 3 folds),0.350 +/- 0.011 (in 3 folds),0.093 +/- 0.008 (in 3 folds),0.121 +/- 0.013 (in 3 folds),0.313,0.350,0.094,0.122,9546430,0,9546430,0.0,False
dummy_stratified,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.354 +/- 0.014 (in 3 folds),0.354 +/- 0.014 (in 3 folds),0.000 +/- 0.000 (in 3 folds),-0.000 +/- 0.001 (in 3 folds),0.354,0.354,0.000,0.000,9546430,0,9546430,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.504 +/- 0.027 (in 3 folds),0.504 +/- 0.027 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.504,0.504,0.000,0.000,9546430,0,9546430,0.0,True


2023-01-03 20:04:54,015 - analyze_sequence_model.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease_all_demographics_present, SampleWeightStrategy.ISOTYPE_USAGE -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/sequence_models/BCR/disease_all_demographics_present_sample_weight_strategy_ISOTYPE_USAGE/train_smaller_model -> /users/maximz/code/boyd-immune-repertoire-classification/out/unirep_fine_tuned/sequence_models/BCR/disease_all_demographics_present_sample_weight_strategy_ISOTYPE_USAGE


********************************************************************************


...

2023-01-04 02:38:38,878 - analyze_sequence_model.ipynb - INFO - GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only, SampleWeightStrategy.ISOTYPE_USAGE -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/sequence_models/TCR/sex_healthy_only_sample_weight_strategy_ISOTYPE_USAGE/train_smaller_model -> /users/maximz/code/boyd-immune-repertoire-classification/out/unirep_fine_tuned/sequence_models/TCR/sex_healthy_only_sample_weight_strategy_ISOTYPE_USAGE


********************************************************************************


# GeneLocus.TCR, TargetObsColumnEnum.sex_healthy_only, SampleWeightStrategy.ISOTYPE_USAGE

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_multiclass,0.510 +/- 0.010 (in 3 folds),0.510 +/- 0.010 (in 3 folds),0.586 +/- 0.130 (in 3 folds),0.586 +/- 0.130 (in 3 folds),0.509 +/- 0.006 (in 3 folds),0.014 +/- 0.014 (in 3 folds),0.509,0.013,6474391,0,6474391,0.0,False
dummy_stratified,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.579 +/- 0.137 (in 3 folds),0.579 +/- 0.137 (in 3 folds),0.506 +/- 0.014 (in 3 folds),-0.000 +/- 0.001 (in 3 folds),0.506,-0.017,6474391,0,6474391,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.579 +/- 0.137 (in 3 folds),0.579 +/- 0.137 (in 3 folds),0.579 +/- 0.137 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.577,0.000,6474391,0,6474391,0.0,True


********************************************************************************
